In [1]:
using IntervalArithmetic
using LaTeXStrings
using Plots

In [2]:
include("./method2/index.jl")
include("./intervalPCM/index.jl")
include("./twofoldInterval/index.jl")
include("./twofoldIntervalPCM/index.jl")

print()

In [3]:
function laTeXMatrix(
        pattern,
        size::Tuple{Integer, Integer})::String
    m, n = size

    mat = "\\begin{bmatrix}"
    for i = 1:m, j = 1:n
        mat *= pattern(i, j)
        if j == n
            if i != m
                mat *= " \\\\ "
            end
        else
            mat *= " & "
        end
    end
    mat *= "\\end{bmatrix}"

    return mat
end

laTeXMatrix (generic function with 1 method)

In [4]:
function method2(A₁::Matrix{Interval{T}}, A₂::Matrix{Interval{T}}) where {T <: Real}
    _, n = size(A₁)

    εᶿ = solveIndividualCancellingLP_m2([A₁, A₂])
    result = solveApproximationLP_m2([A₁, A₂], εᶿ)
    resultₛₜᵣ = approximationLPResultLaTeXString_m2(result)

    println("W^- = %$(resultₛₜᵣ.W⁻)"); println()
    println("W^+ = %$(resultₛₜᵣ.W⁺)"); println()
    println("""
        $(laTeXMatrix((i,j) -> "\\varepsilon_$i^{\\text{L}-}", (n, 1))) = %$(resultₛₜᵣ.εᴸ⁻), ~~
        $(laTeXMatrix((i,j) -> "\\varepsilon_$i^{\\text{U}-}", (n, 1))) = %$(resultₛₜᵣ.εᵁ⁻), ~~
        $(laTeXMatrix((i,j) -> "\\varepsilon_$i^{\\text{L}+}", (n, 1))) = %$(resultₛₜᵣ.εᴸ⁺), ~~
        $(laTeXMatrix((i,j) -> "\\varepsilon_$i^{\\text{U}+}", (n, 1))) = %$(resultₛₜᵣ.εᵁ⁺)
        """); println()

    Â = importance2TwofoldIntervalPCM_m2(result)

    return Â
end

method2 (generic function with 1 method)

# example 1

In [5]:
A₁ = [
    1..1 1..2 (2/3)..(4/3) 1..(4/3)
    (1/2)..1 1..1 (4/9)..1 (2/3)..1
    (3/4)..(3/2) 1..(9/4) 1..1 1..(3/2)
    (3/4)..1 1..(3/2) (2/3)..1 1..1
]
A₂ = A₁

Â = method2(A₁, A₂)

println("\\mathscr{A}^2 = $(twofoldIntervalMatrixLaTeXString(Â))")

W^- = %\begin{bmatrix}\left[ 0.235, 0.314 \right]\\\left[ 0.157, 0.235 \right]\\\left[ 0.235, 0.353 \right]\\\left[ 0.235, 0.235 \right]\end{bmatrix}

W^+ = %\begin{bmatrix}\left[ 0.235, 0.314 \right]\\\left[ 0.157, 0.235 \right]\\\left[ 0.235, 0.353 \right]\\\left[ 0.235, 0.235 \right]\end{bmatrix}

\begin{bmatrix}\varepsilon_1^{\text{L}-} \\ \varepsilon_2^{\text{L}-} \\ \varepsilon_3^{\text{L}-} \\ \varepsilon_4^{\text{L}-}\end{bmatrix} = %\begin{bmatrix} -0.0 \\ 0.0 \\ 0.0 \\ 0.0 \end{bmatrix}, ~~
\begin{bmatrix}\varepsilon_1^{\text{U}-} \\ \varepsilon_2^{\text{U}-} \\ \varepsilon_3^{\text{U}-} \\ \varepsilon_4^{\text{U}-}\end{bmatrix} = %\begin{bmatrix} 0.0 \\ 0.0 \\ 0.0 \\ 0.0 \end{bmatrix}, ~~
\begin{bmatrix}\varepsilon_1^{\text{L}+} \\ \varepsilon_2^{\text{L}+} \\ \varepsilon_3^{\text{L}+} \\ \varepsilon_4^{\text{L}+}\end{bmatrix} = %\begin{bmatrix} 0.0 \\ 0.0 \\ 0.0 \\ 0.0 \end{bmatrix}, ~~
\begin{bmatrix}\varepsilon_1^{\text{U}+} \\ \varepsilon_2^{\text{U}+} \\ \varepsilon_3^{

# example 2

In [6]:
A₁ = [
    1..1 1..2 (2/3)..(4/3) 1..(4/3)
    (1/2)..1 1..1 (4/9)..1 (2/3)..1
    (3/4)..(3/2) 1..(9/4) 1..1 1..(3/2)
    (3/4)..1 1..(3/2) (2/3)..1 1..1
]
_, n = size(A₁)
A₂ = fill(1..1, (n, n))
for i = 1:n, j = 1:n
    if i == j continue end
    A₂[i,j] = (A₁[i,j].lo * 4/5)..(A₁[i,j].hi * 5/4)
end

Â = method2(A₁, A₂)

println("\\mathscr{A}^2 = $(twofoldIntervalMatrixLaTeXString(Â))")

W^- = %\begin{bmatrix}\left[ 0.235, 0.314 \right]\\\left[ 0.157, 0.235 \right]\\\left[ 0.235, 0.353 \right]\\\left[ 0.235, 0.235 \right]\end{bmatrix}

W^+ = %\begin{bmatrix}\left[ 0.206, 0.343 \right]\\\left[ 0.137, 0.258 \right]\\\left[ 0.206, 0.386 \right]\\\left[ 0.206, 0.258 \right]\end{bmatrix}

\begin{bmatrix}\varepsilon_1^{\text{L}-} \\ \varepsilon_2^{\text{L}-} \\ \varepsilon_3^{\text{L}-} \\ \varepsilon_4^{\text{L}-}\end{bmatrix} = %\begin{bmatrix} -0.0 \\ 0.0 \\ 0.0 \\ 0.0 \end{bmatrix}, ~~
\begin{bmatrix}\varepsilon_1^{\text{U}-} \\ \varepsilon_2^{\text{U}-} \\ \varepsilon_3^{\text{U}-} \\ \varepsilon_4^{\text{U}-}\end{bmatrix} = %\begin{bmatrix} 0.0 \\ 0.0 \\ 0.0 \\ 0.0 \end{bmatrix}, ~~
\begin{bmatrix}\varepsilon_1^{\text{L}+} \\ \varepsilon_2^{\text{L}+} \\ \varepsilon_3^{\text{L}+} \\ \varepsilon_4^{\text{L}+}\end{bmatrix} = %\begin{bmatrix} 0.0 \\ 0.0 \\ 0.0 \\ 0.0 \end{bmatrix}, ~~
\begin{bmatrix}\varepsilon_1^{\text{U}+} \\ \varepsilon_2^{\text{U}+} \\ \varepsilon_3^{

<!-- ともに弱い整合性を持ち， $A_{1ij} \subseteq A_{2ij}$

```
A₁ = [
    1..1 2..5 2..4 2..3
    (1/5)..(1/2) 1..1 (1/5)..4 (1/5)..3
    (1/4)..(1/2) (1/4)..5 1..1 (1/4)..3
    (1/3)..(1/2) (1/3)..5 (1/3)..4 1..1
]
A₂ = fill(1..1, size(A₁))
_, n = size(A₁)
for i = 1:n, j = 1:n
    if i == j continue end
    A₂[i,j] = (A₁[i,j].lo * 1/1.1)..(A₁[i,j].hi * 1.1)
end

display(L"A_1 = %$(intervalMatrixLaTeXString(A₁))")
display(L"A_2 = %$(intervalMatrixLaTeXString(A₂))")

display(isConsistentIntervalPCM(A₁))
display(isWeaklyConsistentIntervalPCM(A₁))
display(isConsistentIntervalPCM(A₂))
display(isWeaklyConsistentIntervalPCM(A₂))

plotAll(A₁, A₂)
``` -->

# example 3

In [7]:
A = [
    1 2 4 4
    1/2 1 2 2
    1/4 1/2 1 1
    1/4 1/2 1 1
]

A₁ = randamizedIntervalPCM(A, 1234, 3e-2)
A₂ = randamizedIntervalPCM(A, 134, 3e-2)

Â = method2(A₁, A₂)

println("\\mathscr{A}^2 = $(twofoldIntervalMatrixLaTeXString(Â))")

W^- = %\begin{bmatrix}\left[ 0.5, 0.5 \right]\\\left[ 0.249, 0.25 \right]\\\left[ 0.124, 0.126 \right]\\\left[ 0.124, 0.126 \right]\end{bmatrix}

W^+ = %\begin{bmatrix}\left[ 0.5, 0.501 \right]\\\left[ 0.246, 0.253 \right]\\\left[ 0.122, 0.127 \right]\\\left[ 0.124, 0.127 \right]\end{bmatrix}

\begin{bmatrix}\varepsilon_1^{\text{L}-} \\ \varepsilon_2^{\text{L}-} \\ \varepsilon_3^{\text{L}-} \\ \varepsilon_4^{\text{L}-}\end{bmatrix} = %\begin{bmatrix} 0.0 \\ 0.0 \\ 0.0 \\ 0.0 \end{bmatrix}, ~~
\begin{bmatrix}\varepsilon_1^{\text{U}-} \\ \varepsilon_2^{\text{U}-} \\ \varepsilon_3^{\text{U}-} \\ \varepsilon_4^{\text{U}-}\end{bmatrix} = %\begin{bmatrix} 0.0 \\ 0.0 \\ 0.0 \\ 0.0 \end{bmatrix}, ~~
\begin{bmatrix}\varepsilon_1^{\text{L}+} \\ \varepsilon_2^{\text{L}+} \\ \varepsilon_3^{\text{L}+} \\ \varepsilon_4^{\text{L}+}\end{bmatrix} = %\begin{bmatrix} 0.0 \\ 0.0 \\ 0.0 \\ 0.0 \end{bmatrix}, ~~
\begin{bmatrix}\varepsilon_1^{\text{U}+} \\ \varepsilon_2^{\text{U}+} \\ \varepsilon_3^{\text{U}

# example 4

In [8]:
A = [
    1 2 3 10
    1/2 1 3 8
    1/3 1/3 1 3
    1/10 1/8 1/3 1
]

A₁ = randamizedIntervalPCM(A, 124, 4e-1)
A₂ = randamizedIntervalPCM(A, 411, 4e-1)

Â = method2(A₁, A₂)

println("\\mathscr{A}^2 = $(twofoldIntervalMatrixLaTeXString(Â))")

W^- = %\begin{bmatrix}\left[ 0.5, 0.506 \right]\\\left[ 0.31, 0.323 \right]\\\left[ 0.136, 0.136 \right]\\\left[ 0.041, 0.048 \right]\end{bmatrix}

W^+ = %\begin{bmatrix}\left[ 0.5, 0.522 \right]\\\left[ 0.209, 0.354 \right]\\\left[ 0.097, 0.224 \right]\\\left[ 0.033, 0.061 \right]\end{bmatrix}

\begin{bmatrix}\varepsilon_1^{\text{L}-} \\ \varepsilon_2^{\text{L}-} \\ \varepsilon_3^{\text{L}-} \\ \varepsilon_4^{\text{L}-}\end{bmatrix} = %\begin{bmatrix} 0.0 \\ 0.0 \\ 0.0 \\ 0.0 \end{bmatrix}, ~~
\begin{bmatrix}\varepsilon_1^{\text{U}-} \\ \varepsilon_2^{\text{U}-} \\ \varepsilon_3^{\text{U}-} \\ \varepsilon_4^{\text{U}-}\end{bmatrix} = %\begin{bmatrix} 0.0 \\ 0.0 \\ 0.0 \\ 0.0 \end{bmatrix}, ~~
\begin{bmatrix}\varepsilon_1^{\text{L}+} \\ \varepsilon_2^{\text{L}+} \\ \varepsilon_3^{\text{L}+} \\ \varepsilon_4^{\text{L}+}\end{bmatrix} = %\begin{bmatrix} 0.0 \\ 0.0 \\ 0.0 \\ 0.0 \end{bmatrix}, ~~
\begin{bmatrix}\varepsilon_1^{\text{U}+} \\ \varepsilon_2^{\text{U}+} \\ \varepsilon_3^{\text{